In [1]:
import os

import numpy as np
import pandas as pd

from helper import calculate_iqr_range, get_datatype_mapping_for_reduction, reduce_dataset_size

# set global variables
DATA_DIR = "../data/dataset"
max_std = 402198.1853209468
min_std = 0.0

# set maximum column displayable at once
pd.options.display.max_columns = 122

# load the dataset
df = pd.read_csv(os.path.join(DATA_DIR, "train_data.csv"))
df = reduce_dataset_size(df)
df.info()
df.head()

Reducing dataset size
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184506 entries, 0 to 184505
Columns: 122 entries, SK_ID_CURR to TARGET
dtypes: float32(51), float64(13), int16(3), int32(1), int64(1), object(16), uint32(1), uint8(36)
memory usage: 86.9+ MB


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,Cash loans,F,N,Y,1,67500.0,227520.0,11065.5,180000.0,Family,Commercial associate,Secondary / secondary special,Married,House / apartment,0.018634,-17210,-774,-5580,-755,NaN,1,1,1,1,0,0,Sales staff,3.0,2,2,WEDNESDAY,12,0,0,0,0,0,0,Business Entity Type 3,NaN,0.264354,NaN,0.0124,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0095,NaN,0.000,0.0126,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0099,NaN,0.000,0.0125,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0096,NaN,0.0000,NaN,block of flats,0.0074,Mixed,No,0.0,0.0,0.0,0.0,-150.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,Cash loans,M,Y,Y,1,247500.0,1882372.5,65560.5,1719000.0,Unaccompanied,Working,Higher education,Civil marriage,House / apartment,0.018850,-18640,-96,-5711,-2175,25.0,1,1,0,1,0,1,Managers,3.0,2,2,TUESDAY,10,0,0,0,0,1,1,Business Entity Type 3,0.581064,0.172251,0.686382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-444.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,Cash loans,M,Y,Y,1,180000.0,101880.0,10827.0,90000.0,"Spouse, partner",Working,Higher education,Married,House / apartment,0.022625,-14649,-6130,-1776,-4568,9.0,1,1,1,1,0,0,Laborers,3.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 4,NaN,0.636218,0.540654,0.1979,0.1169,0.9841,0.7824,0.1119,0.16,0.0690,0.4583,0.5,0.1587,0.1614,0.1033,0.0,0.188,0.2017,0.1213,0.9841,0.7909,0.113,0.1611,0.0690,0.4583,0.5,0.1623,0.1763,0.1076,0.0,0.199,0.1999,0.1169,0.9841,0.7853,0.1127,0.16,0.0690,0.4583,0.5,0.1615,0.1642,0.1051,0.0,0.1919,reg oper account,block of flats,0.1789,Panel,No,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,

In [2]:
numerical_columns = list(df.select_dtypes(exclude=["object"]).columns)
print(f"Number of numerical columns: {len(numerical_columns)}\n" )

df[numerical_columns]

Number of numerical columns: 106



,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,1,67500.0,227520.0,11065.5,180000.0,0.018634,-17210,-774,-5580,-755,NaN,1,1,1,1,0,0,3.0,2,2,12,0,0,0,0,0,0,NaN,0.264354,NaN,0.0124,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0095,NaN,0.0000,0.0126,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0099,NaN,0.000,0.0125,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0096,NaN,0.0000,0.0074,0.0,0.0,0.0,0.0,-150.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,1,247500.0,1882372.5,65560.5,1719000.0,0.018850,-18640,-96,-5711,-2175,25.0,1,1,0,1,0,1,3.0,2,2,10,0,0,0,0,1,1,0.581064,0.172251,0.686382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-444.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,1,180000.0,101880.0,10827.0,90000.0,0.022625,-14649,-6130,-1776,-4568,9.0,1,1,1,1,0,0,3.0,2,2,14,0,0,0,0,0,0,NaN,0.636218,0.540654,0.1979,0.1169,0.9841,0.7824,0.1119,0.1600,0.0690,0.4583,0.5000,0.1587,0.1614,0.1033,0.0,0.1880,0.2017,0.1213,0.9841,0.7909,0.1130,0.1611,0.0690,0.4583,0.5000,0.1623,0.1763,0.1076,0.0,0.199,0.1999,0.1169,0.9841,0.7853,0.1127,0.16,0.0690,0.4583,0.5000,0.1615,0.1642,0.1051,0.0,0.1919,0.1789,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0,0
3,305311,0,81000.0,405000.0,20677.5,405000.0,0.035792,-12762,-5891,-6430,-4618,NaN,1,1,1,1,1,0,1.0,2,2,18,0,0,0,0,1,1,NaN,0.599924,0.337673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,7.0,1.0,-1671.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0
4,414121,0,157500.0,888840.0,29506.5,675000.0,0.018801,-16329,-345,-1413,-4624,NaN,1,1,0,1,0,0,2.0,2,2,9,0,0,0,0,1,1,0.476511,0.312779,0.819318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [18]:
def col_info(column_name):
    print(f"{column_name}")
    print(f"Std: {df[column_name].std()}")
    print(f"Min std: {min_std}")
    print(f"Max std:  {max_std}\n")
    
    print(f"Missing value(s): {df[column_name].isna().sum()}\n")

    unique_values = np.sort(df[column_name].unique())
    print(f"Unique values: {len(unique_values)}")
    for unique_value in unique_values:
        print(unique_value, end=", ")

    print("\n")
    print(df[column_name].value_counts())
    
    print("\n")
    iqr_range, lower_bound, upper_bound = calculate_iqr_range(
                df[column_name],
                scaled_factor=1.7,
                percentile_range=(25, 75)
            )
    print(lower_bound, upper_bound)
    index_of_outliers = df[
                (df[column_name] > upper_bound) | (df[column_name] < lower_bound)].index
    print(f"Number of outlier(s): {len(index_of_outliers)}")
    return

In [19]:
column_name = "CNT_CHILDREN"
col_info(column_name)

CNT_CHILDREN
Std: 0.7196143256712833
Min std: 0.0
Max std:  402198.1853209468

Missing value(s): 0

Unique values: 14
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 19, 

0     129082
1      36984
2      15938
3       2190
4        237
5         48
6         12
7          5
14         2
12         2
8          2
9          2
10         1
19         1
Name: CNT_CHILDREN, dtype: int64


-1.7 2.7
Number of outlier(s): 2502


In [7]:
index_of_outliers

NameError: name 'index_of_outliers' is not defined

In [20]:
new = df[column_name].replace([3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 19], 3)

In [23]:
new.value_counts()

0    129082
1     36984
2     15938
3      2502
Name: CNT_CHILDREN, dtype: int64